
#Vamos fazer um projeto de LDA com noticias em portugues 

In [4]:
#!pip install pyLDAvis

In [1]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

Populating the interactive namespace from numpy and matplotlib


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#workdir_path = '/content/drive/My Drive/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
#os.chdir(workdir_path)

In [3]:
data = pd.read_csv('materias.csv')

In [4]:
data.dropna(subset = ["conteudo_noticia"], inplace=True)

In [5]:
data.head()

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia


##vamos testar o LDA, pegaremos só o conteudo das noticias

In [6]:
docs = array(data['conteudo_noticia'])
docs[1]

'\n\tA presidente Dilma Rousseff afirmou nesta sexta-feira (24), durante discurso no Fórum Econômico Mundial em Davos, na Suíça, que é\xa0 "apressada" a tese de que as economias dos países emergentes vão perder dinamismo com o fim da crise financeira mundial. Segundo a presidente, apesar de as economias desenvolvidas darem claros sinais de recuperação, as emergentes vão continuar desempenhando um "papel estratégico". O discurso de Dilma durou aproximadamente 30 minutos.\n\t\xa0"Ainda que as economias desenvolvidas mostrem claros sinais de recuperação, as emergentes continuarão a desempenhar papel estratégico. Estamos falando de países com grandes capacidade de investimento e de ampliação do consumo, com demanda por infraestrutura social, urbana, energia, petróleo, investimentos industriais e agrícolas [...] É apressada a tese de que após a crise as economias emergentes serão menos dinâmicas. Lá estão grandes oportunidades", afirmou a presidente.\n\tAinda segundo Dilma, os países emerge

## Primeiro pré-processar deixando todo em minuscula e tokenizar o texto

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/LeonardoLins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = str(docs[idx]).lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [9]:
docs = docs_preprocessor(docs)

### **Computar bigramas e trigramas :**
Sine tópicos são muito semelhantes, o que os faria distingui-los são frases ao invés de palavras únicas / individuais.

In [10]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/opt/anaconda3/lib/python3.8/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
print(trigram)

Phrases<1707445 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


### Remover Tokens comuns e pouco comuns**

In [12]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 99667
Number of unique words after removing rare and common words: 26722


Eliminando as palavras comuns e raras, acabamos com apenas cerca de 6% das palavras.

** Vetorizar dados: **
A primeira etapa é obter uma representação por trás das palavras de cada documento.
Passos
1-converter um corpus
doc2bow:converter documento (uma lista de palavras) no formato de saco de palavras = lista de (token_id, token_count) 2-tuplas. Cada palavra é considerada uma string tokenizada e normalizada (codificada em unicode ou em utf8). Nenhum outro pré-processamento é feito nas palavras do documento; aplique tokenização, lematização etc. antes de chamar esse método.


In [13]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [14]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 26722
Number of documents: 10103




Com o corpus de bag of words, podemos prosseguir para aprender nosso modelo de tópico a partir dos documentos.

# Entrenando LDA

In [15]:
from gensim.models import LdaModel

In [16]:
# Set training parameters.
num_topics = 6
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 12min 16s, sys: 3.58 s, total: 12min 20s
Wall time: 12min 25s


In [17]:
model.show_topics(num_topics,20)

[(0,
  '0.007*"clube" + 0.005*"jogador" + 0.005*"flamengo" + 0.004*"jogos" + 0.004*"partida" + 0.004*"copa" + 0.004*"pontos" + 0.004*"gols" + 0.004*"jogadores" + 0.004*"vitória" + 0.004*"temporada" + 0.003*"palmeiras" + 0.003*"corinthian" + 0.003*"campo" + 0.003*"técnico" + 0.003*"campeonato" + 0.003*"atacante" + 0.003*"santos" + 0.003*"bola" + 0.003*"rubro_negro"'),
 (1,
  '0.007*"federal" + 0.007*"ministro" + 0.005*"governo" + 0.005*"bolsonaro" + 0.004*"tribunal" + 0.004*"justiça" + 0.004*"decisão" + 0.004*"supremo_tribunal" + 0.004*"temer" + 0.004*"ministério" + 0.004*"nesta_quarta" + 0.003*"ministério_público" + 0.003*"câmara" + 0.003*"presidente_michel" + 0.003*"supremo" + 0.003*"proposta" + 0.003*"processo" + 0.003*"nacional" + 0.003*"pedido" + 0.003*"defesa"'),
 (2,
  '0.007*"empresa" + 0.006*"dado" + 0.005*"rede_social" + 0.005*"você" + 0.005*"segurança" + 0.005*"rede" + 0.005*"facebook" + 0.004*"seus" + 0.004*"conta" + 0.004*"apple" + 0.004*"estados_unidos" + 0.004*"site" + 0.

# Como escolher a quantidade de tópicos?
__LDA__ é uma técnica não supervisionada, o que significa que não sabemos antes de executar o modelo quantos tópicos existem em nosso corpus. A coerência do tópico é uma das principais técnicas utilizadas para desestimar o número de tópicos.

In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
pyLDAvis.__version__

'3.3.1'

In [19]:
gensimvis.prepare(model, corpus, dictionary)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.166924  0.094072       1        1  24.531349
5      0.051947 -0.178080       2        1  21.644609
1     -0.183002  0.133524       3        1  17.385869
3     -0.165076  0.162865       4        1  15.297556
2     -0.158304 -0.237610       5        1  10.935380
4      0.287511  0.025229       6        1  10.205238, topic_info=           Term         Freq        Total Category  logprob  loglift
3572       luta  3141.000000  3141.000000  Default  30.0000  30.0000
5673      clube  4954.000000  4954.000000  Default  29.0000  29.0000
3259   aparelho  2720.000000  2720.000000  Default  28.0000  28.0000
563    ministro  3455.000000  3455.000000  Default  27.0000  27.0000
24      bilhões  2948.000000  2948.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
5379    vitória   848.197767  3504.285280   Topic6  -5.8852   0.8636
2890  principal   712.038197  1787.956296   Topic6  -6.0602   1.3616
5381   vitórias   620.045808  1135.406222   Topic6  -6.1985   1.6773
4610  conseguiu   607.711219  1492.743081   Topic6  -6.2186   1.3836
3294     sábado   628.902996  2347.023495   Topic6  -6.1843   0.9653

[353 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
648        1  0.020717       acesso
648        3  0.030626       acesso
648        5  0.947595       acesso
1775       2  0.917073         acho
1775       6  0.082471         acho
...      ...       ...          ...
24677      5  0.999137        vírus
26558      5  0.999650  vírus_envie
5387       6  1.000406      warrior
22898      5  0.999946     whatsapp
640        4  0.999095       índice

[709 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 2, 4, 3, 5])

** O que vemos aqui? **

** O painel esquerdo **, rotulado Mapa de distância intertópica, os círculos representam diferentes tópicos e a distância entre eles. Tópicos semelhantes aparecem mais próximos e tópicos diferentes mais distantes.
O tamanho relativo do círculo de um tópico no gráfico corresponde à frequência relativa do tópico no corpus.
Um tópico individual pode ser selecionado para um exame mais detalhado clicando em seu círculo ou inserindo seu número na caixa "tópico selecionado" no canto superior esquerdo.
 
** O painel direito ** inclui o gráfico de barras dos 30 principais termos. Quando nenhum tópico é selecionado no gráfico à esquerda, o gráfico de barras mostra os 30 termos mais "salientes" no corpus. A saliência de um termo é uma medida de quão frequente o termo é no corpus e quão "distinto" ele é na distinção entre diferentes tópicos.
Selecionar cada tópico à direita modifica o gráfico de barras para mostrar os termos "relevantes" para o tópico selecionado.
A relevância é definida como no rodapé 2 e pode ser ajustada pelo parâmetro $ \ lambda $, menor $ \ lambda $ dá maior peso à distinção do termo, enquanto $ \ lambda $ s maior corresponde à probabilidade da ocorrência do termo por tópicos.

Portanto, para ter uma noção melhor dos termos por tópico, usaremos $ \ lambda $ = 0.